<a href="https://colab.research.google.com/github/mmcenta/le-net-beginner/blob/master/notebooks/modern_le_net_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

# Modern LeNet in TensorFlow
This notebook is an implementation of the "Hello World" of ConvNets: the LeNet 5 architecture, evaluated on the MNIST dataset.

This is intended as a tool for beginners to get familiar with TensorFlow as well as a training exercise for myself.

This notebooks draws heavily from the TensorFlow turorials in style and topic order.

Now that the introduction is out of the way, let's build our network

## Load the data
Let's load the MNIST dataset from torchvision. We will also apply a transformation pipeline that converts the images to tensors and normalizes them to the [-1.0, 1.0] range.

First, we import the necessary modules and classes.

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

Then we load the MNIST dataset and prepare it.

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = (x_train / 255.0), x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

Finally, we shuffle and batch the data.

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(64)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

## Define the model

Next we need to define the model we'll be using. We will be using a convolutional neural network deeply inspired on the classic LeNet architecture proposed in the "Gradient-based learning applied to document recognition" paper by LeCun et al.

In [0]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D

class ModernLeNet(Model):
  def __init__(self):
    super(ModernLeNet, self).__init__()
    # Convolutional layers
    self.conv1 = Conv2D(6, 5, padding='same', activation='relu')
    self.maxpool1 = MaxPool2D(2)
    self.conv2 = Conv2D(16, 5, padding='same', activation='relu')
    self.maxpool2 = MaxPool2D(2)

    # Fully connected layers
    self.flatten = Flatten()
    self.fc1 = Dense(120, activation='relu')
    self.fc2 = Dense(84, activation='relu')
    self.fc3 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.fc2(x)
    return self.fc3(x)

# Create an instance of the model
model = ModernLeNet()


We will use the cross entropy loss for this clasification task, as well as the Adam optimizer with default parameters.

In [0]:
loss_object = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()